In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdmolops, rdMolTransforms, ChemicalFeatures
import os

In [2]:
fdefName = os.path.join("/Users/andicuko/anaconda2/pkgs/rdkit-2018.09.3.0-py27h65625ec_1/share/RDKit/Data",'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train = train[train["molecule_name"] != "dsgdb9nsd_059827"] #to skip "dsgdb9nsd_059827" it seems to be a transition state

In [6]:
#there are many duplicates to be removed
train = train.round(decimals=1)
train.drop_duplicates(subset=["molecule_name","type", "scalar_coupling_constant"], inplace=True)

In [7]:
train = pd.concat([train, test], sort=False)

In [8]:
u = train["molecule_name"].unique() # each molecule
g = train.groupby("molecule_name")

In [9]:
electronegativity = {"H":2.20,"C": 2.55,"N": 3.04,"O": 3.44,"F": 3.98}

In [10]:
# this take a while to finish!
# Here, I go through each molecule and for each molecule I see all atoms pare and 
# I extract a series of features from them.
%%time
property_dict = {
    "molecule_name": [],
    "atom_index_0": [],
    "atom_index_1": [],
    "type": [],
    "atom_1_type": [],
    "atom_1_hybridization": [],
    "pi_bonds": [],
    "graph_distance": [],
    "graph_smile": [],
    "angle":[],
    "dihedral": [],
    "sum_electronegativity_inbetwen":[],
    "sum_electronegativity_neghb":[],
    "donor_groups_in_neighb": [],
    "aceptor_groups_in_neighb": [],
    "posIonizable_groups_in_neighb": [],
    "aromatic_groups_in_neighb": [],
    "hydrophobe_groups_in_neighb": [],
    "lumpedHydrophobe_groups_in_neighb": [],
    "negIonizable_groups_in_neighb": [], 
}

wrong_mol_files = []
count = 0
for mol_n in u:
    count += 1
    if count % 10000 ==0:
        print(count) # just to keep track to where we are in the proccess
    mol = Chem.MolFromMolFile("structures_mol/"+mol_n+".mol", removeHs=False)
    if mol is None:
        mol = Chem.SDMolSupplier("structures_sdf/"+mol_n+".sdf", removeHs=False)[0]
    if mol is None:
        # to keep track of problematic molecules
        print(mol_n)
        wrong_mol_files.append(mol_n)
    else:
        try:
            a = g.get_group(mol_n)
            pairs = zip(a.atom_index_0.values, a.atom_index_1.values, a.type.values)
            dist_matrix = Chem.rdmolops.Get3DDistanceMatrix(mol)    
            for pair in pairs:
                atom_index_0 = pair[0]
                atom_index_1 = pair[1]
                J_type = pair[2]
                atom_1 = mol.GetAtomWithIdx(pair[1])
                atom_1_type = atom_1.GetSymbol()
                atom_1_hybridization = str(atom_1.GetHybridization())
                sp = rdmolops.GetShortestPath(mol,atom_index_0,atom_index_1)
                graph_dist = 0.0
                pi_bonds = 0.0
                for i in range(len(sp[:-1])):
                    graph_dist += dist_matrix[sp[i]][sp[i+1]]
                    b = str(mol.GetBondBetweenAtoms(sp[i],sp[i+1]).GetBondType())
                    if b=="DOUBLE":
                        pi_bonds += 1
                    if b=="TRIPLE":
                        pi_bonds += 2
                    if b=="AROMATIC":
                        pi_bonds += 1.5  
                conf=mol.GetConformer(-1)    
                if len(sp) == 2: angle = 0.0; dihedral = 0.0    
                if len(sp) == 3: 
                    angle = rdMolTransforms.GetAngleRad(conf,sp[0],sp[1],sp[2])  
                    dihedral = 0.0
                if len(sp) == 3: angle = rdMolTransforms.GetAngleRad(conf,sp[0],sp[1],sp[2])  
                if len(sp) == 4: 
                    dihedral = rdMolTransforms.GetDihedralRad(conf, sp[0], sp[1], sp[2], sp[3])
                    angle1 = rdMolTransforms.GetAngleRad(conf,sp[0],sp[1],sp[2]) 
                    angle2 = rdMolTransforms.GetAngleRad(conf,sp[1],sp[2],sp[3])
                    angle = (angle1 + angle2)/2.0
    
                property_dict["molecule_name"].append(mol_n)
                property_dict["atom_index_0"].append(atom_index_0)
                property_dict["atom_index_1"].append(atom_index_1)
                property_dict["type"].append(J_type)
                property_dict["atom_1_type"].append(atom_1_type)
                property_dict["atom_1_hybridization"].append(atom_1_hybridization)
                property_dict["pi_bonds"].append(pi_bonds)
                property_dict["angle"].append(angle)
                property_dict["dihedral"].append(dihedral)    
                property_dict["graph_distance"].append(graph_dist)
                
                atoms_inbetween = "".join([mol.GetAtomWithIdx(x).GetSymbol() for x in sp])
                property_dict["graph_smile"].append(atoms_inbetween)
                
                sum_electronegativity_inbetwen = sum([electronegativity[mol.GetAtomWithIdx(x).GetSymbol()] for x in sp])
                sum_electronegativity_neghb = 0.0
                neigh = list(atom_1.GetNeighbors())
                
                #molecule features
                mol_features = factory.GetFeaturesForMol(mol) 
                familyfeat_to_atoms = {}
                atoms_to_familyfeat = {}
                for f in mol_features:
                    atom_ids = f.GetAtomIds()
                    familyfeat_to_atoms[f.GetFamily()] = [i for i in atom_ids]
                unique_atoms_id = list(set([item for sublist in familyfeat_to_atoms.values() for item in sublist]))
                for i in unique_atoms_id:
                    atoms_to_familyfeat[i] = [k for k,v in familyfeat_to_atoms.iteritems() if i in v]
    
                neigh_numb = len(neigh)
                donor_groups_in_neighb              = 0
                aceptor_groups_in_neighb            = 0
                posIonizable_groups_in_neighb       = 0
                aromatic_groups_in_neighb           = 0
                hydrophobe_groups_in_neighb         = 0
                lumpedHydrophobe_groups_in_neighb   = 0
                negIonizable_groups_in_neighb       = 0
                for n in neigh:
                    n_id = n.GetIdx()
                    if n_id != sp[0]:
                        sum_electronegativity_neghb += electronegativity[n.GetSymbol()]
                        if n_id in atoms_to_familyfeat.keys(): 
                            if "Donor"           in atoms_to_familyfeat[i] : donor_groups_in_neighb            += 1 
                            if "Acceptor"        in atoms_to_familyfeat[i] : aceptor_groups_in_neighb          += 1 
                            if "PosIonizable"    in atoms_to_familyfeat[i] : posIonizable_groups_in_neighb     += 1 
                            if "Aromatic"        in atoms_to_familyfeat[i] : aromatic_groups_in_neighb         += 1 
                            if "Hydrophobe"      in atoms_to_familyfeat[i] : hydrophobe_groups_in_neighb       += 1 
                            if "LumpedHydrophobe"in atoms_to_familyfeat[i] : lumpedHydrophobe_groups_in_neighb += 1 
                            if "NegIonizable"    in atoms_to_familyfeat[i] : negIonizable_groups_in_neighb     += 1 
    
                property_dict["donor_groups_in_neighb"].append(donor_groups_in_neighb)
                property_dict["aceptor_groups_in_neighb"].append(aceptor_groups_in_neighb)
                property_dict["posIonizable_groups_in_neighb"].append(posIonizable_groups_in_neighb)
                property_dict["aromatic_groups_in_neighb"].append(aromatic_groups_in_neighb)
                property_dict["hydrophobe_groups_in_neighb"].append(hydrophobe_groups_in_neighb)
                property_dict["lumpedHydrophobe_groups_in_neighb"].append(lumpedHydrophobe_groups_in_neighb)
                property_dict["negIonizable_groups_in_neighb"].append(negIonizable_groups_in_neighb)
                    
                property_dict["sum_electronegativity_inbetwen"].append(sum_electronegativity_inbetwen)
                property_dict["sum_electronegativity_neghb"].append(sum_electronegativity_neghb)
        except:
            print count

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
CPU times: user 7h 38min 2s, sys: 3min 38s, total: 7h 41min 41s
Wall time: 7h 50min 38s


In [11]:
columns = [
    'molecule_name',
    'atom_index_1',
    'atom_index_0',
    'type',
    'atom_1_type',
    'atom_1_hybridization',
    'pi_bonds',    
    'graph_distance',
    'graph_smile',
    'angle',
    'dihedral',
    "sum_electronegativity_inbetwen",
    "sum_electronegativity_neghb",
    "donor_groups_in_neighb"           ,
    "aceptor_groups_in_neighb"         ,
    "posIonizable_groups_in_neighb"    ,
    "aromatic_groups_in_neighb"        ,
    "hydrophobe_groups_in_neighb"      ,
    "lumpedHydrophobe_groups_in_neighb",
    "negIonizable_groups_in_neighb"    ,
]

prop = pd.DataFrame(property_dict, columns=columns)

In [12]:
prop['sigma_bonds'] = prop["type"].apply(lambda x: x[0])

In [14]:
train = pd.merge(train, prop, on=["molecule_name", "atom_index_1", "atom_index_0", "type"])

In [16]:
# saving the work
train.to_csv("featurized_train.csv", index=False)